In [12]:
import pandas as pd
from pandas import DataFrame as df
import os

In [13]:
patinfo = pd.read_csv('./my_data/sepsis_merged_data.csv')
concat_data = pd.read_csv('./my_data_concat/all_parameters.csv')

In [14]:
concat_data = concat_data.drop(['Unnamed: 0'], axis=1)
concat_data = concat_data.dropna(subset=['stay_id'],axis=0)

In [19]:
concat_data = concat_data.drop(['stay_id'], axis=1)

In [17]:
sepsis_pat = patinfo[patinfo['y']==1]

In [27]:
concat_data['chartdate'] = concat_data['t_time'].map(lambda x: x[:10])

In [28]:
concat_data

,subject_id,hadm_id,t_time,respir_rate,heart_rate,blood_oxy_satur,sys_blood_pressure,temperature,pH_value,wbc_count,CO2_part_press,dias_blood_pressure,chartdate
0,10000032,29079034,2180-07-23 14:00:00,22.5,92.0,97.5,89.5,98.7,NaN,NaN,NaN,53.5,2180-07-23
1,10000032,29079034,2180-07-23 15:00:00,23.0,94.0,97.0,88.0,NaN,NaN,NaN,NaN,56.0,2180-07-23
2,10000032,29079034,2180-07-23 16:00:00,21.0,105.0,94.0,91.0,NaN,NaN,NaN,NaN,55.0,2180-07-23
3,10000032,29079034,2180-07-23 17:00:00,20.0,97.0,95.0,95.0,98.7,NaN,NaN,NaN,58.0,2180-07-23
4,10000032,29079034,2180-07-23 18:00:00,21.0,100.0,95.0,86.0,NaN,NaN,NaN,NaN,53.0,2180-07-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7841830,15612570,26152435,2114-04-05 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.0,2114-04-05
7841831,16721241,22649116,2183-08-30 05:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,125.0,2183-08-30
7841832,18986646,25231593,2146-12-09 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.0,2146-12-09
7841833,19635953,29856471,2180-11-23 11:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78.0,2180-11-23


In [29]:
result = pd.merge(sepsis_pat, concat_data, on=['subject_id', 'hadm_id', 'chartdate'], how='left')

In [32]:
result = result.dropna(subset=['t_time'])

In [36]:
sum(list(result.isna().sum()))/9

23745.666666666668

In [37]:
sub_id = list(set(result.subject_id.values))

In [39]:
len_sub = [len(result[result['subject_id']==i]) for i in sub_id]

In [40]:
mean_nan = [sum(list(result[result['subject_id']==i].isna().sum()))/9 for i in sub_id]

In [64]:
cal_data = df({'subject_id':sub_id, 'len_sub':len_sub, 'nan_mean':mean_nan})

In [68]:
cal_data['nan'] = cal_data['nan_mean']/cal_data['len_sub']

In [76]:
cal_data.sort_values(by=['len_sub'], ascending=False)

,subject_id,len_sub,nan_mean,nan
484,11281568.0,221,88.000000,0.398190
1583,19509298.0,180,71.444444,0.396914
1485,13594409.0,106,40.333333,0.380503
627,16909817.0,103,38.444444,0.373247
2714,16497723.0,91,36.222222,0.398046
...,...,...,...,...
1601,13258859.0,1,0.555556,0.555556
1749,15004194.0,1,0.333333,0.333333
1608,15749248.0,1,0.888889,0.888889
2497,16554461.0,1,0.444444,0.444444


In [70]:
cal_data.sort_values(by=['nan'])

,subject_id,len_sub,nan_mean,nan
1521,10014610.0,1,0.111111,0.111111
2479,14211509.0,2,0.222222,0.111111
965,10267064.0,1,0.111111,0.111111
2755,12270760.0,1,0.111111,0.111111
1387,13479459.0,1,0.111111,0.111111
...,...,...,...,...
1608,15749248.0,1,0.888889,0.888889
2874,10976739.0,1,0.888889,0.888889
1875,14594997.0,3,2.666667,0.888889
749,15533907.0,11,9.777778,0.888889


In [77]:
result.isna().sum()

subject_id                 0
hadm_id                    0
seq_num                    0
y                          0
chartdate                  0
age                        0
t_time                     0
respir_rate             1181
heart_rate              1023
blood_oxy_satur         1851
sys_blood_pressure      1743
temperature            41117
pH_value               53607
wbc_count              55808
CO2_part_press         55637
dias_blood_pressure     1744
dtype: int64

In [78]:
cal_data[cal_data['len_sub']>100].sort_values(by=['nan'])

,subject_id,len_sub,nan_mean,nan
627,16909817.0,103,38.444444,0.373247
1485,13594409.0,106,40.333333,0.380503
1583,19509298.0,180,71.444444,0.396914
484,11281568.0,221,88.000000,0.398190


In [79]:
result[result['subject_id']==11281568]

,subject_id,hadm_id,seq_num,y,chartdate,age,t_time,respir_rate,heart_rate,blood_oxy_satur,sys_blood_pressure,temperature,pH_value,wbc_count,CO2_part_press,dias_blood_pressure
22025,11281568.0,25148783.0,1.0,1,2127-02-18,55,2127-02-18 00:00:00,34.0,74.0,88.0,92.0,98.3,NaN,NaN,NaN,58.0
22026,11281568.0,25148783.0,1.0,1,2127-02-18,55,2127-02-18 01:00:00,27.0,66.0,91.0,92.0,NaN,NaN,NaN,NaN,54.0
22027,11281568.0,25148783.0,1.0,1,2127-02-18,55,2127-02-18 02:00:00,34.0,63.0,100.0,99.0,NaN,NaN,8.0,NaN,52.0
22028,11281568.0,25148783.0,1.0,1,2127-02-18,55,2127-02-18 03:00:00,25.0,65.0,100.0,115.0,NaN,NaN,NaN,NaN,64.0
22029,11281568.0,25148783.0,1.0,1,2127-02-18,55,2127-02-18 04:00:00,24.0,57.0,99.0,90.0,98.2,NaN,NaN,NaN,62.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63117,11281568.0,28411808.0,2.0,1,2128-05-30,56,2128-05-30 19:00:00,33.0,121.0,97.0,99.0,NaN,NaN,NaN,NaN,63.0
63118,11281568.0,28411808.0,2.0,1,2128-05-30,56,2128-05-30 20:00:00,32.0,122.0,96.0,109.0,100.8,NaN,NaN,NaN,68.0
63119,11281568.0,28411808.0,2.0,1,2128-05-30,56,2128-05-30 21:00:00,37.0,123.0,96.0,115.0,NaN,NaN,NaN,NaN,69.0
63120,11281568.0,28411808.0,2.0,1,2128-05-30,56,2128-05-30 22:00:00,31.0,126.0,96.0,121.0,99.3,NaN,NaN,NaN,72.0
